# 目标检测 Mask R-CNN

本文主要参考了pytorch官方文档：[TORCHVISION OBJECT DETECTION FINETUNING TUTORIAL](https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html)，了解目标检测的实践，后面会有能用到本专业内容上的。关于算法方面的基础只是可以参考elks项目中的记录。

在Penn-Fudan数据库中对行人检测和分割的预训练Mask R-CNN模型进行微调。它包含170张图像，其中包含345个行人实例，我们将用它来说明如何在torchvision中使用新功能，以便在自定义数据集上训练实例细分模型。

## 定义数据集

用于训练对象检测，实例细分和人员关键点检测的参考脚本可轻松支持添加新的自定义数据集。数据集应继承自标准 torch.utils.data.Dataset类，并实现__len__和 __getitem__。

我们唯一需要的特性是数据集__getitem__ 应该返回：

- 图像：大小为(H, W)的PIL的图像 
- 目标：包含以下字段的dict：
    - boxes (FloatTensor[N, 4])：N 个边界框的坐标，格式为[x0, y0, x1, y1]，范围从 0到W，0到H
    - labels (Int64Tensor[N])：每个边界框的标签
    - image_id (Int64Tensor[1])：图像标识符。它在数据集中的所有图像之间应该是唯一的，并在评估过程中使用
    - area (Tensor[N])：边界框的面积。在使用COCO度量进行评估时，可使用此值来区分小，中和大boxes之间的度量得分。
    - iscrowd (UInt8Tensor[N])：iscrowd = True的实例在 evaluation 期间将被忽略。
    - （可选）masks (UInt8Tensor[N, H, W])：每个对象的分割蒙版
    - （可选）keypoints (FloatTensor[N, K, 3])：对于N个对象中的每个对象，它包含格式为[x, y, visibility]的K个关键点，用于定义对象。visibility= 0表示关键点不可见。请注意，对于数据扩充，翻转关键点的概念取决于数据表示形式，您可能应该使用references/detection/transforms.py来适应新的关键点表示形式
    
如果模型返回上述方法，则它们将使其适用于训练和评估，并将使用pycocotools中的评估脚本。

此外，如果要在训练过程中使用宽高比分组（以便每个批次仅包含具有相似长宽比的图像），则建议实现一种get_height_and_width 方法，该方法可返回图像的高度和宽度。如果未提供此方法，那么将通过 __getitem__ 查询数据集的所有元素，这会将图像加载到内存中，使得比提供自定义方法时要慢。

那么接下来就为PennFudan编写自定义数据集。

先[下载](https://www.cis.upenn.edu/~jshi/ped_html/PennFudanPed.zip)并解压缩zip文件，放到 5-nn-example\cnns\data 文件夹下。

In [ ]:
import os
import numpy as np
import torch
from PIL import Image


class PennFudanDataset(object):
    def __init__(self, root, transforms):
        self.root = root
        self.transforms = transforms
        # load all image files, sorting them to
        # ensure that they are aligned
        self.imgs = list(sorted(os.listdir(os.path.join(root, "PNGImages"))))
        self.masks = list(sorted(os.listdir(os.path.join(root, "PedMasks"))))

    def __getitem__(self, idx):
        # load images ad masks
        img_path = os.path.join(self.root, "PNGImages", self.imgs[idx])
        mask_path = os.path.join(self.root, "PedMasks", self.masks[idx])
        img = Image.open(img_path).convert("RGB")
        # note that we haven't converted the mask to RGB,
        # because each color corresponds to a different instance
        # with 0 being background
        mask = Image.open(mask_path)
        # convert the PIL Image into a numpy array
        mask = np.array(mask)
        # instances are encoded as different colors
        obj_ids = np.unique(mask)
        # first id is the background, so remove it
        obj_ids = obj_ids[1:]

        # split the color-encoded mask into a set
        # of binary masks
        masks = mask == obj_ids[:, None, None]

        # get bounding box coordinates for each mask
        num_objs = len(obj_ids)
        boxes = []
        for i in range(num_objs):
            pos = np.where(masks[i])
            xmin = np.min(pos[1])
            xmax = np.max(pos[1])
            ymin = np.min(pos[0])
            ymax = np.max(pos[0])
            boxes.append([xmin, ymin, xmax, ymax])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)
        # there is only one class
        labels = torch.ones((num_objs,), dtype=torch.int64)
        masks = torch.as_tensor(masks, dtype=torch.uint8)

        image_id = torch.tensor([idx])
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        # suppose all instances are not crowd
        iscrowd = torch.zeros((num_objs,), dtype=torch.int64)

        target = {}
        target["boxes"] = boxes
        target["labels"] = labels
        target["masks"] = masks
        target["image_id"] = image_id
        target["area"] = area
        target["iscrowd"] = iscrowd

        if self.transforms is not None:
            img, target = self.transforms(img, target)

        return img, target

    def __len__(self):
        return len(self.imgs)

## 定义模型

在本教程中，我们将使用基于 Faster R-CNN的Mask R-CNN。Faster R-CNN是预测 图像中潜在对象的边界框bounding boxes 和类分数class scores 的模型。

Mask R-CNN在Faster R-CNN中增加了一个分支，可以预测每个实例的分割掩码。更多简介内容可以参考elks项目中的记录。

在两种常见情况下，可能要修改Torchvision modelzoo中的可用模型之一。首先是当我们想从预训练模型开始，然后微调最后一层。另一个是当我们要用另一个模型替换主干时（例如，为了更快的预测）。

接下来让我们看看如何做。

1. 通过预训练模型进行微调：假设您想从在COCO上进行过预训练的模型开始，并希望针对您的特定班级对其进行微调。这是一种可行的方法。

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained on COCO
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 2  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

2. Modifying the model to add a different backbone

In [ ]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# load a pre-trained model for classification and return
# only the features
backbone = torchvision.models.mobilenet_v2(pretrained=True).features
# FasterRCNN needs to know the number of
# output channels in a backbone. For mobilenet_v2, it's 1280
# so we need to add it here
backbone.out_channels = 1280

# let's make the RPN generate 5 x 3 anchors per spatial
# location, with 5 different sizes and 3 different aspect
# ratios. We have a Tuple[Tuple[int]] because each feature
# map could potentially have different sizes and
# aspect ratios
anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

# let's define what are the feature maps that we will
# use to perform the region of interest cropping, as well as
# the size of the crop after rescaling.
# if your backbone returns a Tensor, featmap_names is expected to
# be [0]. More generally, the backbone should return an
# OrderedDict[Tensor], and in featmap_names you can choose which
# feature maps to use.
roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=[0],
                                                output_size=7,
                                                sampling_ratio=2)

# put the pieces together inside a FasterRCNN model
model = FasterRCNN(backbone,
                   num_classes=2,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)